In [51]:
class Crystal_output:
    """
    Crystal_output class - Add frequency-related attributes.
    Currently supported attribute:
        self.file: list of string, __init__, Information of output file.
        self.volume: float, get_volume, Lattice volume. Unit: A^3
        self.nmode: int, get_mode_block, Number of vibration modes.
        self.frequency: numpy float array, get_mode, Harmonic vibrational
                        frequency. Unit: THz
        self.eigenvalue: numpy float array, get_mode, Eigenvalues of harmonic
                         modes.Unit: Hartree^2
    """
    # Original attribute
    def __init__(self, output_name):
        # output_name: name of the output file

        import sys
        import re

        self.name = output_name

        # Check if the file exists
        try:
            if output_name[-3:] != 'out' and output_name[-4:] != 'outp':
                output_name = output_name+'.out'
            file = open(output_name, 'r')
            self.data = file.readlines()
            file.close()
        except:
            print('EXITING: a .out file needs to be specified')
            sys.exit(1)

        # Check the calculation converged
        self.converged = False

        for i, line in enumerate(self.data[::-1]):
            if re.match(r'^ EEEEEEEEEE TERMINATION', line):
                self.converged = True
                # This is the end of output
                self.eoo = len(self.data)-1-i
                break

        if self.converged == False:
            self.eoo = len(self.data)
            
    def get_qpoint(self):
        """
        Get the qpoints at which the phonon frequency is calculated.
        Input:
            -
        Output:
            self.nqpoint, int, Number of k points.
            self.qpoint, nq * 3 numpy float array, fractional coordinates of qpoints.
        """
        import numpy as np
        import re
        
        self.nqpoint = 0
        self.qpoint = np.array([], dtype=float)

        for i, line in enumerate(self.data):
            if re.search('EXPRESSED IN UNITS        OF DENOMINATOR', line):
                shrink = int(line.strip().split()[-1])
                
            if re.search('DISPERSION K POINT NUMBER', line):
                coord = np.array(line.strip().split()[7:10], dtype=float)
                self.qpoint = np.append(self.qpoint, coord / shrink)
                self.nqpoint += 1
        
        self.qpoint = np.reshape(self.qpoint, (-1, 3))
        if self.nqpoint == 0:
            self.nqpoint = 1
            self.qpoint = np.array([0, 0, 0], dtype=float)
            
        return self.nqpoint, self.qpoint

    def get_mode(self):
        """
        Get corresponding vibrational frequencies and for all modes and
        compute the total number of vibration modes (natoms * 3).

        Input:
            -
        Output:
            self.nmode, int, Number of vibration modes
            self.frequency: nmode * nqpoint numpy float array, Harmonic vibrational
                        frequency. Unit: THz
        """
        import numpy as np
        import re
        
        if not hasattr(self, 'nqpoint'):
            self.get_qpoint()
        
        self.frequency = np.array([], dtype=float)

        countline = 0
        while countline < len(self.data):
            is_freq = False
            if re.search('DISPERSION K POINT NUMBER', self.data[countline]):
                countline += 2
                is_freq = True
            
            if re.search('MODES         EIGV          FREQUENCIES     IRREP',
                         self.data[countline]):
                countline += 2
                is_freq = True

            while self.data[countline].strip() and is_freq:
                nm_a = int(self.data[countline].strip().split()[0].strip('-'))
                nm_b = int(self.data[countline].strip().split()[1])
                freq = float(self.data[countline].strip().split()[4])

                for mode in range(nm_a, nm_b + 1):
                    self.frequency = np.append(self.frequency, freq)

                countline += 1
                
            countline += 1

        self.frequency = np.reshape(self.frequency, (-1, self.nqpoint))
        self.nmode = len(self.frequency[0])

        return self.nmode, self.frequency


In [52]:
import re

p6freq = Crystal_output('12x12nointer.out')
p6freq.get_mode()

(90,
 array([[-1.50000e-03,  7.00000e-04,  7.00000e-04,  2.49403e+01,
          4.96607e+01,  4.96607e+01,  5.13000e-02,  1.34330e+00,
          2.11440e+00,  2.49303e+01,  4.96060e+01,  4.97087e+01,
          1.66100e-01,  2.68760e+00,  4.20990e+00,  2.49008e+01,
          4.94509e+01,  4.98426e+01,  3.37000e-01,  4.03270e+00,
          6.27350e+00,  2.48534e+01,  4.92198e+01,  5.00348e+01,
          5.70000e-01,  5.37600e+00,  8.29810e+00,  2.47904e+01,
          4.89434e+01,  5.02482e+01,  8.65100e-01,  6.71340e+00,
          1.02824e+01,  2.47143e+01,  4.86489e+01,  5.04449e+01,
          1.22120e+00,  8.03960e+00,  1.22264e+01,  2.46275e+01,
          4.83516e+01,  5.05942e+01,  1.63730e+00,  9.35030e+00,
          1.41273e+01,  2.45314e+01,  4.80527e+01,  5.06768e+01,
          2.11170e+00,  1.06420e+01,  1.59782e+01,  2.44263e+01,
          4.77424e+01,  5.06852e+01,  2.64240e+00,  1.19131e+01,
          1.77693e+01,  2.43111e+01,  4.74089e+01,  5.06201e+01,
          3.22620e+0